# **Accessing HPC Cluster and Job Submission**

**Section 1: Section A — SSH Access & Environment Sanity Check(Login Node)**

In [ ]:
#hello_cluster.py
import os
import platform
import sys
print("Hello from HPC cluster!")
print("User:",os.getenv("USER"))
print("Host:",platform.node())
print("Python:",sys.version)
print("cores(logical):",os.cpu_count())

Hello from HPC cluster!
User: None
Host: 297736f8c55d
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
cores(logical): 2


**Section 2: Section B — First Batch Job: Serial Python Script via Slurm/PBS**

In [ ]:
#serial_baseline.py
import time,os,platform,sys
def work(n=5_00_000):
  s=0.0
  for i in range(n):
    s+=(i%7)*0.123456
  return s
if __name__=='__main__':
  print("===Job Info===")
print("Host",platform.node())
print("User",os.getenv("USER"))
print("Python",sys.version)
print("===work===")
t0=time.perf_counter()
result=work()
t1=time.perf_counter()
print("result={result.:6f}|Time={(t1-t0).:3f}s")


===Job Info===
Host 297736f8c55d
User None
Python 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
===work===
result={result.:6f}|Time={(t1-t0).:3f}s


In [ ]:
!salloc -p short -N 1 -n 1 -c 1 -t 00:10:00
!hostname


/bin/bash: line 1: salloc: command not found
297736f8c55d


**Section 3: Section C — Interactive Compute Session (salloc/srun or qsub -I)**

In [ ]:
import os,time,random

def estimate_pi(n_samples,seed=42):
    random.seed(seed)
    inside=0
    for _ in range(n_samples):
        x=random.random()
        y=random.random()
        inside+=(x*x+y*y<=1.0)
    return 4.0*inside/n_samples

if __name__=="__main__":
    tid=int(os.getenv("SLURM_ARRAY_TASK_ID",os.getenv("PBS_ARRAYID","0")))
    sizes=[50_000,100_000,200_000,300_000,400_000,800_000]
    n=sizes[tid%len(sizes)]
    t0=time.perf_counter()
    pi=estimate_pi(n,seed=100+tid)
    t1=time.perf_counter()
    print(f"TaskID={tid} N={n} pi\u2248{pi:.6f} Time={(t1-t0):.3f}s")


TaskID=0 N=50000 pi≈3.141360 Time=0.012s


**Section 4: Section D — Job Arrays for Parameter Sweep(Multiple Serial Runs)**

In [ ]:
import time,random,cProfile,pstats,io,tracemalloc

def monte_carlo_pi(n,seed=42):
 random.seed(seed)
 inside=0
 for _ in range(n):
  x=random.random()
  y=random.random()
  if x*x+y*y<=1.0:
   inside+=1
 return 4.0*inside/n

def main():
 N=2_000_000
 tracemalloc.start()
 pr=cProfile.Profile()
 pr.enable()
 t0=time.perf_counter()
 pi_est=monte_carlo_pi(N)
 t1=time.perf_counter()
 pr.disable()
 current,peak=tracemalloc.get_traced_memory()
 tracemalloc.stop()
 print(f"Samples={N}")
 print(f"Estimated Pi={pi_est:.6f}")
 print(f"Time:{t1-t0:.3f}s")
 print(f"Current/Peak memory:{current/1e6:.2f}MB/{peak/1e6:.2f}MB")
 s_buf=io.StringIO()
 ps=pstats.Stats(pr,stream=s_buf).sort_stats("cumtime")
 ps.print_stats(10)
 print("\n---cProfile (Top 10)---")
 print(s_buf.getvalue())

if __name__=="__main__":
 main()


Samples=2000000
Estimated Pi=3.141602
Time:6.082s
Current/Peak memory:0.01MB/0.01MB

---cProfile (Top 10)---
         4000020 function calls (4000019 primitive calls) in 6.083 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    3.974    0.795    5.028    1.006 {built-in method time.sleep}
  4000000    1.270    0.000    1.270    0.000 {method 'random' of '_random.Random' objects}
      2/1    0.839    0.419    0.615    0.615 /tmp/ipython-input-889023621.py:3(monte_carlo_pi)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 /usr/lib/python3.12/random.py:135(seed)
        6    0.000    0.000    0.000    0.000 {built-in method posix.getppid}
        1    0.000    0.000    0.000    0.000 {function Random.seed at 0x78909a822a20}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        2    0.000

**Section 5: Section E — Modules/Venv, Scratch I/O, and Resource Flags**

In [ ]:
import time,random,cProfile,pstats,io,tracemalloc

def gen_particles(n,seed=42):
 random.seed(seed)
 return [(random.random(),random.random()) for _ in range(n)]

def pairwise_kernel(pos):
 n=len(pos)
 acc=[0.0]*n
 for i in range(n):
  xi,yi=pos[i]
  s=0.0
  for j in range(n):
   if i!=j:
    xj,yj=pos[j]
    dx=xi-xj
    dy=yi-yj
    r2=dx*dx+dy*dy+1e-12
    s+=1.0/r2
  acc[i]=s
 return acc

def main():
 N=800
 pos=gen_particles(N)
 tracemalloc.start()
 pr=cProfile.Profile()
 pr.enable()
 t0=time.perf_counter()
 acc=pairwise_kernel(pos)
 t1=time.perf_counter()
 pr.disable()
 current,peak=tracemalloc.get_traced_memory()
 tracemalloc.stop()
 print(f"Particles={N}")
 print(f"Sample acc[0]={acc[0]:.6f}")
 print(f"Time:{t1-t0:.3f}s")
 print(f"Current/Peak memory:{current/1e6:.2f}MB/{peak/1e6:.2f}MB")
 s_buf=io.StringIO()
 ps=pstats.Stats(pr,stream=s_buf).sort_stats("cumtime")
 ps.print_stats(10)
 print("\n---cProfile (Top 10)---")
 print(s_buf.getvalue())

if __name__=="__main__":
 main()


Particles=800
Sample acc[0]=112064.380774
Time:3.054s
Current/Peak memory:0.03MB/0.03MB

---cProfile (Top 10)---
         31 function calls in 2.872 seconds

   Ordered by: cumulative time
   List reduced from 24 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.872    2.872 /usr/lib/python3.12/asyncio/base_events.py:1922(_run_once)
        1    0.185    0.185    2.872    2.872 /usr/lib/python3.12/selectors.py:451(select)
        2    2.014    1.007    2.014    1.007 {built-in method time.sleep}
        1    0.672    0.672    0.672    0.672 /tmp/ipython-input-2483977469.py:7(pairwise_kernel)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 /usr/lib/python3.12/asyncio/events.py:86(_run)
        1    0.000    0.000    0.000    0.000 {method 'run' of '_contextvars.Context' objects}
        1    0.000    0.000    0